In [1]:
# import functools
# import operator
# import pathlib
import gzip

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.ticker as mtick

#import seaborn as sns
import torch

import config
# import plot_utils
# import contacts
import utils

In [2]:
model_prefix = "DHFR_incl_main_kit_taq"
DATADIR = config.WORKING_DIR

In [3]:
L = config.L; q=20
main_params = utils.load_torch_tensor(f"{DATADIR}/{model_prefix}_{L}_main_params.pt.gz")
main_params_exp = torch.exp(main_params)
main_params_prob= main_params_exp / main_params_exp.sum(1, keepdim=True)

del_indices_pos = torch.argmin(main_params_prob, dim=1)
# indices for the flattend matrix
del_indices =  del_indices_pos + np.array(list(range(0,L*q,q)))

h_ia_r = torch.load(f"../working/{model_prefix}_h_i_a.pt")
e_ijab_r = torch.load(f"../working/{model_prefix}_e_i_a_j_b.pt")

eta = torch.load(f"../working/{model_prefix}_eta.pt")
J = torch.load(f"../working/{model_prefix}_J.pt")

# Energy (from Gaussian Density)

In [4]:
def msa_seq(seq):
    L = config.L
    q = 20
    x=torch.zeros(L*q)
    nzind = seq+ np.array(list(range(0,config.L*q,q)))
    x[nzind]=1
    return(x)

In [5]:
seq = config.WT_AA_TENSOR

seq_msa = msa_seq(seq)
seq_msa = torch.tensor(np.delete(seq_msa,del_indices,axis=0)) # reduced
-sum(eta * seq_msa - 1/2*seq_msa*torch.matmul(J,seq_msa)) # E(x) = -(eta'x -1/2 x'Jx)

C:\Users\hps5320\AppData\Local\Temp/ipykernel_21048/2267806709.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_msa = torch.tensor(np.delete(seq_msa,del_indices,axis=0)) # reduced


tensor(0.7909)

# Energy (MRF canonical params)

In [6]:
def energy_main(h,e,aa):
    field_sum = 0
    for i in range(config.L):
        field_sum += h[i,aa[i]] 
        
    return(-field_sum)

def energy_int(e,aa):
    cpl_sum =0
    for i in range(config.L):
        for j in range(i+1,config.L):
            cpl_sum += e[i,aa[i],j,aa[j]]
            # cpl_sum += e[j,aa[j],i,aa[i]]
    return(-cpl_sum)

In [7]:
energy_main(h_ia_r,e_ijab_r,seq) + energy_int(e_ijab_r, seq)

tensor(0.7909, dtype=torch.float64)

# Energy (using utils)

In [8]:
utils.calc_energy_pytorch(seq, h_ia_r, e_ijab_r)

tensor(0.7909)